# Vertex AI Workshop

## Table of Contents
* [Introduction](#Introduction)
* [Setup Environment](#I.Setup-Environment)
    * [Install Libraries](#Install-Libraries)
    * [Import Libraries](#Import-Libraries)
    * [Define Constants and Variables](#Define-Constants-and-Variables)
    * [Initialize Vertex AI](#Initialize-Vertex-AI)
* [Download Workshop Data](#Download-Workshop-Data)
* [Build our Vertex Pipeline](#II.-Vertex-Pipelines)
    * [Create Pipeline Components](#Create-Pipeline-Components)
* [Run & Explore Experiments](#Run-and-explore-experiments)
* [Get Predictions](#Get-Predictions)
    * [Get Online Predictions](#Get-Online-Predictions)
    * [Get Batch Predictions](#Get-Batch-Predictions)
* [Next Steps](#Next-Steps)

<img src=img/custom-kfp-pipeline.png>

In [ ]:
#!pip install -U google-cloud-pipeline-components

## Introduction

In this workshop ... blablabla
TODO

## Setup Environment

### Install Libraries

In [ ]:
# !pip3 install --user google-cloud-aiplatform==1.7.0 --upgrade -q --no-warn-conflicts
# !pip3 install --user kfp==1.8.9 google-cloud-pipeline-components==1.0.7 -q --no-warn-conflicts
# !pip3 install --user tensorflow -q --no-warn-conflicts
# !pip install --user tf-models-official -q --no-warn-conflicts

### Import libraries

In [ ]:
import os
import shutil
import logging
from datetime import datetime

# Import the Vertex AI Python SDK 
from google.cloud import aiplatform as vertexai
from google.cloud.aiplatform import pipeline_jobs
import google.auth
import google.cloud.aiplatform as vertex
from google.cloud import storage
import pandas as pd
import numpy as np
from datetime import datetime
from google.cloud import bigquery
#from google.cloud import bigquery

# google_cloud_pipeline_components includes pre-built KFP components for interfacing with Vertex AIservices.
from google_cloud_pipeline_components import aiplatform as gcc_aip


# kfp sdk, to create the Vertex AI Pipelines
from kfp.v2 import compiler, dsl
from kfp.v2.dsl import pipeline

from typing import NamedTuple
from kfp.v2 import dsl
from kfp.v2.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component, 
                        OutputPath,
                        InputPath)
from kfp.v2 import compiler

from typing import NamedTuple
import google.cloud.aiplatform as aip
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import component, Artifact, Dataset, Input, Metrics, Model, Output
from google_cloud_pipeline_components.experimental import vertex_notification_email as gcc_exp

# TensorFlow model building libraries.
import tensorflow as tf

# Re-create the AdamW optimizer used in the original BERT paper.
from official.nlp import optimization  

# Libraries for data and plot model training metrics.
import pandas as pd
import matplotlib.pyplot as plt


# Own Functions
from src.helper import upload_file_to_gcs

In [ ]:
#!pip3 show google_cloud_pipeline_components

In [ ]:
import google.auth
import google.cloud.aiplatform as vertex
import pandas as pd
import numpy as np
from datetime import datetime
from google.cloud import bigquery

_, PROJECT_ID = google.auth.default()
REGION = "us-central1"
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

def get_service_account():
    # Todo: use python client instead.
    service_account = !gcloud iam service-accounts list --filter="Compute Engine default service account"
    service_account = np.array(service_account[1].split())[5]
    SERVICE_ACCOUNT = f"{service_account}"
    return SERVICE_ACCOUNT

SERVICE_ACCOUNT = get_service_account()

In [ ]:
# Create bucket
BUCKET_NAME = f"{PROJECT_ID}-bucket"

# Check if Bucket already exists, if not, then create the bucket
# Imports the Google Cloud client library

# Instantiates a client
storage_client = storage.Client()

# Check if bucket already exists:
buckets = [bucket.name for bucket in storage_client.list_buckets()]
if not BUCKET_NAME in buckets:
    bucket = storage_client.create_bucket(BUCKET_NAME)


## Download Workshop Data

In [ ]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'

column_names = [
    'MPG', 'Cylinders', 'Displacement',
    'Horsepower', 'Weight', 'Acceleration', 'Model Year', 'Origin'
]

raw_dataset = pd.read_csv(
    url,
    names=column_names,
    na_values='?',
    comment='\t',
    sep=' ',
    skipinitialspace=True,
)

raw_dataset.rename(
    columns = {
        'MPG': 'mpg',
        'Cylinders': 'cyl',
        'Displacement': 'dis',
        'Horsepower': 'hp',
        'Weight': 'weight',
        'Acceleration': 'accel',
        'Model Year': 'year',
        'Origin': 'origin'
    }, inplace = True
)

raw_dataset.dropna(inplace=True)
gcs_path = 'data/fuel_data.csv'
raw_dataset.to_csv(header=False, index=False, path_or_buf=gcs_path)

In [ ]:
upload_file_to_gcs(bucket_name=BUCKET_NAME, source_file_name='data/fuel_data.csv', destination_blob_name=gcs_path)

------------------------------------

## Vertex Pipelines

<img src=img/vertex-pipelines.png width=800>

To address your business requirements and get your higher performing model into production to deliver value faster, you will define a pipeline using the [**Kubeflow Pipelines (KFP) V2 SDK**](https://www.kubeflow.org/docs/components/pipelines/sdk/v2/v2-compatibility) to orchestrate the training and deployment of your model on [**Vertex Pipelines**](https://cloud.google.com/vertex-ai/docs/pipelines) below.

The pipeline consists of five components:

* `get_reviews_datasets Custom Component` [(documentation)](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-0.2.0/google_cloud_pipeline_components.aiplatform.html#google_cloud_pipeline_components.aiplatform.CustomContainerTrainingJobRunOp): gets the required data and transform it for the training requirements.

* `train_bert_classifier Custom Component` [(documentation)](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-0.2.0/google_cloud_pipeline_components.aiplatform.html#google_cloud_pipeline_components.aiplatform.CustomContainerTrainingJobRunOp): trains your custom model using a custom component from kfp. This is the same as configuring a Vertex Custom Container Training Job using the Vertex Python SDK you covered in the Vertex AI: Qwik Start lab.

* `classification_model_eval_metrics Custom Component` [(documentation)](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-0.2.0/google_cloud_pipeline_components.aiplatform.html#google_cloud_pipeline_components.aiplatform.CustomContainerTrainingJobRunOp): evaluates the trained model and makes the decision whether to deploy the model and create an endpoint based on a passed threshold.


*  `deploy_model_endpoint Custom Component` [(documentation)](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-0.2.0/google_cloud_pipeline_components.aiplatform.html#google_cloud_pipeline_components.aiplatform.EndpointCreateOp): Creates a Google Cloud Vertex Endpoint resource that maps physical machine resources with your model to enable it to serve online predictions. Online predictions have low latency requirements; providing resources to the model in advance reduces latency. 



### Create Pipeline's Components

### Pipeline Step 1: Create BigQuery Dataset

In [ ]:
@component(
    base_image="python:3.9",
    packages_to_install=['google-cloud-bigquery', 'pandas'],
    output_component_file="create_bq_dataset_op.yml"
)
def create_bq_dataset_op(
    gcs_uri: str,
    project_id: str,
    dataset_name: str,
    table_name: str,
) -> NamedTuple('Outputs',[('bq_dataset_uri', str)]):
    """
    Create a new bucket in the US region with the STANDARD storage class
    Args:
        gcs_uri: gcs uri (gs://...)
        bucket_name: name of the bucket
        region: region or zone
        service_account: service account
    Output:
        table_id:string, Table in BigQuery
    """
    
    from google.cloud import bigquery
    import os

    # Create bigquery table
    bq_client = bigquery.Client(project=project_id)
    dataset_name = dataset_name

    dataset_id = "{}.{}".format(bq_client.project, dataset_name)
    dataset = bigquery.Dataset(dataset_id)
    dataset.location = "US"

    try:
        dataset = bq_client.create_dataset(dataset, timeout=30)
        print("Created dataset {}.{}".format(bq_client.project, dataset.dataset_id))
    except:
        bq_client.delete_dataset(dataset_id, delete_contents=True)
        dataset = bq_client.create_dataset(dataset, timeout=30)

    # Create table
    table_name = table_name
    table_id = f"{dataset_id}.{table_name}"

    job_config = bigquery.LoadJobConfig(
        schema=[
            bigquery.SchemaField("mpg", bigquery.enums.SqlTypeNames.FLOAT64),
            bigquery.SchemaField("cyl", bigquery.enums.SqlTypeNames.INTEGER),
            bigquery.SchemaField("dis", bigquery.enums.SqlTypeNames.FLOAT64),
            bigquery.SchemaField("hp", bigquery.enums.SqlTypeNames.FLOAT64),
            bigquery.SchemaField("weight", bigquery.enums.SqlTypeNames.FLOAT64),
            bigquery.SchemaField("accel", bigquery.enums.SqlTypeNames.FLOAT64),
            bigquery.SchemaField("year", bigquery.enums.SqlTypeNames.INTEGER),
            bigquery.SchemaField("origin", bigquery.enums.SqlTypeNames.INTEGER),
        ], 
        write_disposition="WRITE_TRUNCATE")

    job = bq_client.load_table_from_uri(
        gcs_uri, table_id, job_config=job_config)

    job.result()

    bq_dataset_uri = f"bq://{dataset_id}.{table_name}"
    from collections import namedtuple
    output = namedtuple('Outputs',['bq_dataset_uri'])

    return output(bq_dataset_uri)

### Pipeline Step 2: Prepare train, val & test datasets

In [ ]:
@component(
    base_image="python:3.9",
    packages_to_install= ['google-cloud-bigquery[bqstorage,pandas]'],
    output_component_file="prepare_datasets_op.yml"
)
def prepare_datasets_op(
    bq_dataset: str,
    dataset_train: Output[Dataset],
    dataset_test: Output[Dataset],
)-> NamedTuple("Outputs", [("train_dataset_path", str), ("test_dataset_path", str)]):
    
    # print(f"Input dataset is: {bq_dataset}")
    # print(f"Input dataset is: {bq_dataset.uri}")
    # print(f"Input dataset is: {bq_dataset.location}")
    from google.cloud import bigquery
    import pandas as pd
    import logging
    import os
    
    
    
    bqclient = bigquery.Client()
    
    def download_table(bq_table_uri: str):
        prefix = "bq://"
        if bq_table_uri.startswith(prefix):
            bq_table_uri = bq_table_uri[len(prefix):]

        table = bigquery.TableReference.from_string(bq_table_uri)
        rows = bqclient.list_rows(
            table,
        )
        return rows.to_dataframe(create_bqstorage_client=False)
    
    
    raw_dataset = download_table(bq_dataset)
    raw_dataset.rename(columns = {
        'mpg':'MPG',
        'cyl':'Cylinders',
        'dis':'Displacement',
        'hp': 'Horsepower',
        'weight': 'Weight',
        'accel': 'Acceleration',
        'year': 'Model Year',
        'origin': 'Origin'}, inplace = True)

    # Get data in shape
    dataset = raw_dataset.copy()
    dataset.tail()
    dataset = dataset.dropna()
    dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})
    dataset = pd.get_dummies(dataset, columns=['Origin'], prefix='', prefix_sep='')
    train_dataset = dataset.sample(frac=0.8, random_state=0)
    test_dataset = dataset.drop(train_dataset.index)
    
    train_dataset_path = dataset_train.path
    test_dataset_path = dataset_test.path
    logging.info(f"Dataset Train is be stored in: {dataset_train.path}")
    logging.info(f"Dataset Test is be stored in: {dataset_test.path}")
    
    train_dataset.to_csv(dataset_train.path, index=False)
    test_dataset.to_csv(dataset_train.path, index=False)
    
    return (train_dataset_path, test_dataset_path)
    


### Pipeline Step 3: Train & Evaluate our Machine Learning Model

In [ ]:
@component(
    base_image="tensorflow/tensorflow:2.10.0",
    packages_to_install=['pandas'],
    output_component_file="train_evaluate_model_op.yml"
)
def train_evaluate_model_op(
    dataset_train: Input[Dataset],
    dataset_test: Input[Dataset],
    threshold_dict_str: str,
    pipeline_name: str,
    metrics: Output[Metrics],
    model: Output[Model],
    model_uri: OutputPath(str),
) -> NamedTuple(
        'Outputs',[
            ('loss', float),
            ('val_loss', float),
            ('dep_decision', str),
            ('model_path', str)
        ]):
    
    import os
    import logging
    import tensorflow as tf
    import pandas as pd
    import numpy as np
    from tensorflow import keras
    from tensorflow.keras import layers
    import json
    
    logging.info(f"Train Dataset path {dataset_train.path}")
    logging.info(f"Test Dataset path {dataset_test.path}")
    logging.info(f"train dataset - {dataset_train.path}")
    
    train_dataset = pd.read_csv(dataset_train.path)
    test_dataset = pd.read_csv(dataset_train.path)
    train_features = train_dataset.copy()
    test_features = test_dataset.copy()
    train_labels = train_features.pop('MPG')
    test_labels = test_features.pop('MPG')

    
    # Create model
    normalizer = tf.keras.layers.Normalization(axis=-1)
    normalizer = tf.keras.layers.Normalization(axis=-1)
    normalizer.adapt(np.array(train_features))
    normalizer = tf.keras.layers.Normalization(axis=-1)
    normalizer.adapt(np.array(train_features))
    first = np.array(train_features[:1])
    horsepower = np.array(train_features['Horsepower'])
    horsepower_normalizer = layers.Normalization(input_shape=[1,], axis=None)
    horsepower_normalizer.adapt(horsepower)

    def build_and_compile_model(norm):
        model = keras.Sequential([
            norm,
            layers.Dense(64, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(1)
        ])
        
        metrics_tf = [tf.metrics.MeanAbsoluteError(), tf.metrics.MeanAbsolutePercentageError(), 
               tf.metrics.MeanSquaredError()]
        
        model.compile(loss='mean_absolute_error', 
                      optimizer=tf.keras.optimizers.Adam(0.001),
                      metrics=metrics_tf
                     )
        return model

    dnn_model = build_and_compile_model(normalizer)
    dnn_model.summary()

    history = dnn_model.fit(
        train_features,
        train_labels,
        validation_split=0.2,
        verbose=0, epochs=100
    )

    
    loss_test, mae, mape, mse = dnn_model.evaluate(test_features,
                                                       test_labels,
                                                       verbose=0
                                                      )
    
    logging.info(f"TEST METRICS: {loss_test}, {mae}, {mape}, {mse}")

    # Log metrics
    metrics_training = {metric: values[-1] for metric, values in history.history.items()}
    metrics.log_metric('loss', metrics_training['loss'])
    metrics.log_metric('val_loss', metrics_training['val_loss'])
    metrics.log_metric('test_loss', loss_test)
    metrics.log_metric('test_mae', mae)
    metrics.log_metric('test_mape', mape)
    metrics.log_metric('test_mse', mse)
  
    
    model.metadata['loss'] = metrics_training['loss']
    model.metadata['val_loss'] = metrics_training['val_loss']
    model.metadata['test_loss'] = loss_test
    model.metadata['pipeline'] = pipeline_name

    # Save the model to GCS
    dnn_model.save(model.path)

    threshold_dict = json.loads(threshold_dict_str)
    kpi_decision = float(threshold_dict['mae'])
    
        
    if mae <= kpi_decision:
        dep_decision = "true"
    else:
        dep_decision = "false"
    logging.info(f"deployment decision is {dep_decision}")
    logging.info(f"model will be stored in {model.path}")
    
    model_uri = model.path

    return (metrics_training['loss'], metrics_training['val_loss'], dep_decision, model.path)

### Pipeline Step 4: Upload our Model to Model Registry, Create and Endpoint & Deploy our Model

In [ ]:
@component(
  base_image="python:3.9",
  packages_to_install=['google-cloud-aiplatform'],
  output_component_file="upload_model_op.yml"
)
def deploy_model_endpoint_op(
    model: Input[Model],
    region: str,
    model_name: str,
    pipeline_name: str,
    project_id: str,
    serving_container_image_uri: str,
    machine_type: str,
    model_display_name: str,
    endpoint_display_name:str,
    vertex_model: Output[Artifact]
):
    import logging
    from google.cloud import aiplatform as aip
        
    labels={'pipeline': str(pipeline_name),
            'country': 'germany'}
   
    aip.init(project=project_id, location=region)

    logging.info(f"Model URI {model.uri}")
    logging.info(f"Model Name: {model_name}")
    logging.info(f"Model Labels: {labels}")
    logging.info(f"serving_container_image_uri {serving_container_image_uri}")
    
    def create_endpoint(endpoint_display_name, project, region):
        endpoints = aip.Endpoint.list(
        filter='display_name="{}"'.format(endpoint_display_name),
        order_by='create_time desc',
        project=project, 
        location=region,
        )
        if len(endpoints) > 0:
            endpoint = endpoints[0]  # most recently created
        else:
            endpoint = aiplatform.Endpoint.create(
            display_name=endpoint_display_name, project=project, location=region
        )
        return endpoint
    
    # Create Endpoint     
    endpoint = create_endpoint(endpoint_display_name, project_id, region) 
    
    
    model_upload = aip.Model.upload(  
        display_name=model_name,
        artifact_uri=model.uri,
        description='Regression model for fuel prediction',
        labels=labels,
        serving_container_image_uri=serving_container_image_uri
        )
    
    logging.info(f'Input Endpoint {endpoint}')
    model_deploy = model_upload.deploy(
        machine_type=machine_type, 
        endpoint=endpoint,
        traffic_split={"0": 100},
        deployed_model_display_name=model_display_name,
    )

    # Save data to the output params
    vertex_model.uri = model_deploy.resource_name
    logging.info(model_deploy.resource_name)
    
#

### Build & Compile our Pipeline integrating all components

#### Define some constants

In [ ]:
from datetime import datetime

USER = 'add-your-name-lowercase'
if USER == 'add-your-name-lowercase':
    USER = 'unknown'

USER = 'gabriela'

EXPERIMENT_NAME = "fuel-model"
EXPERIMENT_DESCRIPTION = "Fuel prediction pipeline"
PIPELINE_ROOT = f"gs://{BUCKET_NAME}/pipeline_root/{USER}"
PIPELINE_NAME = f"{EXPERIMENT_NAME}-{TIMESTAMP}"
PIPELINE_PACKAGE_PATH = f'{PIPELINE_NAME}-path.json'


MODEL_NAME = EXPERIMENT_NAME
THRESHOLD_DICT_STR = '{"mae": 10000}'

MANAGED_DATASET_NAME = 'fuel_dataset'
TABLE_NAME = 'main'
MODEL_NAME = 'fuel-prediction'
DATA_GCS_DIR = f"gs://{BUCKET_NAME}/{gcs_path}"


ENDPOINT_DISPLAY_NAME = 'fuel-endpoint'
SERVING_CONTAINER_IMAGE_URI = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-6:latest'
ENDPOINT_MACHINE_TYPE = "n1-standard-4"



LABELS = {
    'creator': USER,
    'workflow': 'fuel-prediction',
    'type': 'regression'}

aip.init(
    project=PROJECT_ID,
    staging_bucket=BUCKET_NAME,
    experiment=EXPERIMENT_NAME,
    experiment_description="Fuel prediction pipeline")

In [ ]:
# PIPELINE
@dsl.pipeline(
    pipeline_root=PIPELINE_ROOT,
    name=PIPELINE_NAME
)
def pipeline(
    gcs_uri: str,
    dataset_name: str,
    threshold_dict_str: str,
    user:str,
    managed_dataset_name: str,
    endpoint_display_name: str,
    endpoint_machine_type:str,
    table_name: str,
    project_id: str,
    pipeline_name: str,
    serving_container_image_uri:str,
    model_name: str,
    region: str):

    # STEP 1:  Create bq table and dataset
    create_bq_dataset_task = create_bq_dataset_op(
        gcs_uri=gcs_uri,
        project_id=project_id,
        dataset_name=dataset_name,
        table_name=table_name
        ).set_caching_options(True) \
        .set_display_name('create-bq-dataset-table-op')
    
    
    
   # STEP 2: Prepare train and test datasets
    prepare_data_op = prepare_datasets_op(
        #bq_dataset=create_dataset_op.outputs['dataset']
        bq_dataset =create_bq_dataset_task.outputs['bq_dataset_uri']
    ).set_caching_options(True) \
        .set_display_name('prepare-datasets-op')

    # STEP 3: Train and evaluate model
    train_evaluate_model_task = train_evaluate_model_op(
        dataset_train = prepare_data_op.outputs['dataset_train'],
        dataset_test = prepare_data_op.outputs['dataset_test'],
        threshold_dict_str=threshold_dict_str,
        pipeline_name=pipeline_name
    ).set_display_name('training-evaluation-job-op')\
        .set_caching_options(True)

    ## Step3: Decision: If model performs according to our threshold, then deploy model and Enp
    with dsl.Condition(
            train_evaluate_model_task.outputs["dep_decision"] == "true",
            name="deploy_decision",
        ):
            # Upload Model, Create a Vertex Endpoint resource and Deploy Model to Endpoint.

            deploy_model_endpoint_op(
                            model=train_evaluate_model_task.outputs['model'],
                            region=region,
                endpoint_display_name=endpoint_display_name,
                            model_name=model_name,
                            pipeline_name=pipeline_name,
                            project_id=project_id,
                machine_type=endpoint_machine_type,
                model_display_name=model_name,
                serving_container_image_uri=serving_container_image_uri
            ).set_display_name('deploy-model-op').set_caching_options(True)
    


In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path=PIPELINE_PACKAGE_PATH
)

### Run Pipeline

In [ ]:
job = aip.PipelineJob(
    display_name=PIPELINE_NAME,
    template_path=PIPELINE_PACKAGE_PATH,
    pipeline_root=PIPELINE_ROOT,
    enable_caching=True,
    labels=LABELS,
    parameter_values={
        'gcs_uri': DATA_GCS_DIR,
        'dataset_name': MANAGED_DATASET_NAME,
        'threshold_dict_str':THRESHOLD_DICT_STR,
        'user': USER,
        'managed_dataset_name': MANAGED_DATASET_NAME,
        'endpoint_display_name': ENDPOINT_DISPLAY_NAME,
        'endpoint_machine_type': ENDPOINT_MACHINE_TYPE,
        'table_name': TABLE_NAME,
        'project_id': PROJECT_ID,
        'pipeline_name': PIPELINE_NAME,
        'serving_container_image_uri': SERVING_CONTAINER_IMAGE_URI,
        'model_name': MODEL_NAME,
        'region': REGION
    }
)


job.submit(experiment=EXPERIMENT_NAME)

## Online Predictions with Deployed Endpoint

Retrieve the `Endpoint` deployed by the pipeline and use it to query your model for online predictions.

Configure the `Endpoint()` function below with the following parameters:

*  `endpoint_name`: A fully-qualified endpoint resource name or endpoint ID. Example: "projects/123/locations/us-central1/endpoints/456" or "456" when project and location are initialized or passed.
*  `project_id`: GCP project.
*  `location`: GCP region.

Call `predict()` to return a prediction for a test review.

In [329]:
ENDPOINT_NAME = '9056039561246801920'

In [330]:
endpoint = vertexai.Endpoint(ENDPOINT_NAME)

In [331]:
prediction = endpoint.predict([4,90.0,75.0,2125.0,14.5,74,0,0,1])

In [332]:
prediction

Prediction(predictions=[[26.7319756]], deployed_model_id='113350852730683392', model_version_id='1', model_resource_name='projects/525292812449/locations/us-central1/models/5413674197773713408', explanations=None)

## Batch Predictions with Created Model

In [ ]:
# Define batch job args
TIMESTAMP =datetime.now().strftime("%Y%m%d%H%M%S") 
batch_job_display_name = "fuel-batch-prediction-job"
gcs_batch_data = ''
instances_format = 'csv'
gcs_dest_results = f''
machine_type = "n1-standard-2"

In [ ]:
!gcloud ai models list --region=us-central1

In [ ]:
MODEL_NAME = ''
MODEL_ID = !(gcloud ai models list --region=$REGION \
           --filter=display_name=$MODEL_NAME)
MODEL_ID = MODEL_ID[2].split(" ")[0]
MODEL_ID

In [ ]:
model_resource_name = f'projects/{PROJECT_ID}/locations/{REGION}/models/{MODEL_ID}'
model_resource_name

In [ ]:
# aiplatform.init(project=project, location=location)
model = aiplatform.Model(model_resource_name)

In [ ]:
batch_prediction_job = model.batch_predict(
        job_display_name=batch_job_display_name,
        instances_format='csv', #json
        gcs_source=[gcs_batch_data],
        gcs_destination_prefix=gcs_dest_results,
        machine_type=machine_type, # must be present      
    )